Final model: the one used to predict instances outside the dataset

Imports

In [9]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import *
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix, accuracy_score
from sklearn import svm, tree
from sklearn.model_selection import KFold,cross_val_score
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.sparse import csr_matrix, hstack

Training

In [10]:
dataset = pd.read_json("drive/My Drive/ML/HW1/processed_dataset.json",lines=True)

vectorizer = CountVectorizer(min_df=0.005)
y_all = dataset['semantic']
X1 = vectorizer.fit_transform(dataset['instructions'])

columns=vectorizer.get_feature_names()

#adding number of edges
edges=dataset['edges']
l=len(edges)
rows=np.arange(0,l)
cols=np.full((l), 0, dtype=int)
X2=csr_matrix((edges, (rows, cols)),shape=(l, 1)).toarray()
X_tmp2=hstack((X1,X2))
columns.append('edges')

#adding cycles(true/false)
cycles=dataset['cycles']
X3=csr_matrix((cycles, (rows, cols)),shape=(l, 1)).toarray()
X_tmp=hstack((X_tmp2,X3))
columns.append('has_cycles')


#adding cyclomatic complexity
cyclo=dataset['cyclomatic']
X4=csr_matrix((cyclo, (rows, cols)),shape=(l, 1)).toarray()
X_all=hstack((X_tmp,X4))
columns.append('cyclomatic_c')

#Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
          test_size=0.2, random_state=15)
id = random.randrange(0,X_train.shape[0])

start=time.time()
#model = MultinomialNB().fit(X_train, y_train)
model = svm.SVC(kernel = 'linear', C=1).fit(X_train, y_train)
t=time.time()-start
print("training running time: "+str(t)+" sec")



training running time: 42.093008279800415 sec


Evaluation

In [11]:
y_pred = model.predict(X_test)
cm=confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
print("accuracy_score: "+str(accuracy_score(y_test,y_pred, normalize=True)))

[[557   0   1   0]
 [  0 922   5   0]
 [  0   2 785   9]
 [  0   1  34 564]]
              precision    recall  f1-score   support

  encryption       1.00      1.00      1.00       558
        math       1.00      0.99      1.00       927
        sort       0.95      0.99      0.97       796
      string       0.98      0.94      0.96       599

    accuracy                           0.98      2880
   macro avg       0.98      0.98      0.98      2880
weighted avg       0.98      0.98      0.98      2880

accuracy_score: 0.9819444444444444


Prediction on the blind test

In [12]:
newfile=open("drive/My Drive/ML/HW1/1796480.txt","w")
blindtest = pd.read_json("drive/My Drive/ML/HW1/processed_blindtest.json",lines=True)

X1=vectorizer.transform(blindtest['instructions'])

#adding number of edges
edges=blindtest['edges']
l=len(edges)
rows=np.arange(0,l)
cols=np.full((l), 0, dtype=int)
X2=csr_matrix((edges, (rows, cols)),shape=(l, 1)).toarray()
X_tmp2=hstack((X1,X2))

#adding cycles(true/false)
cycles=blindtest['cycles']
X3=csr_matrix((cycles, (rows, cols)),shape=(l, 1)).toarray()
X_tmp=hstack((X_tmp2,X3))


#adding cyclomatic complexity
cyclo=blindtest['cyclomatic']
X4=csr_matrix((cyclo, (rows, cols)),shape=(l, 1)).toarray()
X_all=hstack((X_tmp,X4))


predictions=model.predict(X_all)
for pred in predictions:
  newfile.write(pred+'\n')
newfile.close()
print("Results in 1796480.txt!")

Results in 1796480.txt!
